In [1]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
## File parameter
FOLDER_PATH = '/content/drive/MyDrive/music-retrival'
TRAIN_DATA = FOLDER_PATH+ "/data-emotion/set1/spectograms_313x149_Mel_func/"
LABEL_DATA = FOLDER_PATH + "/data-emotion/set1/mean_ratings_set1.xls"

TEST_DATA = FOLDER_PATH+ "/data-emotion/set2/spectograms_313x149_Mel_func/"
EVAL_DATA = FOLDER_PATH + "/data-emotion/set2/mean_ratings_set2.xls"

MODEL_PATH = FOLDER_PATH + '/model/mid2e/'
## DATA PARAMETER
NUM_INSTANCE = 360
TEST_SIZE = 0.2

## Model Parameter
BATCH_SIZE = 8
EPOCHS = 40
MODEL_SAVE = True

In [3]:
def load_data_label(feature_path, label_file_path):
    label_matrix = pd.read_excel(label_file_path, header=0).drop(['Number','TARGET'],axis=1)
    # Load the data from each file and concatenate into a single feature matrix
    feature_matrix = None

    for i in range(1, len(label_matrix) + 1):
        if i < 10:
          i = '00' + str(i)
        elif i < 100:
          i = '0' + str(i)

        file_path = os.path.join(feature_path, str(i) + ".mp3.npy")
        # print("Processed file " + file_path)
        data = np.load(file_path)
        data = [np.expand_dims(data, axis=-1)]
        if feature_matrix is None:
            feature_matrix = data
        else:
            feature_matrix = np.concatenate((feature_matrix, data), axis=0)
    return feature_matrix, label_matrix


In [4]:
 # load feature, label
feature_matrix, label_matrix = load_data_label(TRAIN_DATA, LABEL_DATA)

feature_matrix = feature_matrix.reshape((feature_matrix.shape[0], -1))

# split train, test
train_features, test_features, train_labels, test_labels = train_test_split(feature_matrix, label_matrix, test_size=TEST_SIZE, random_state=42)

In [5]:
train_features.shape

(288, 46637)

In [7]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, make_scorer

In [11]:
cor_df = {'Emotion':[], 'Correlation':[]}
metric = {'Emotion': [], 'MSE': [], 'R2': [], 'MAE': []}

for i in train_labels.columns.values:
  svr = SVR()
  param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf'],
    'epsilon': [0.1, 0.01, 0.001]
}

  # Define the scoring metric (negative mean squared error)
  scoring = make_scorer(mean_squared_error, greater_is_better=False)

  # Perform grid search with cross-validation
  grid_search = GridSearchCV(svr, param_grid, scoring=scoring, cv=5)
  grid_search.fit(train_features, train_labels[i])
  # svr.fit(train_features, train_labels[i])
  best_svr = grid_search.best_estimator_
  y_pred = best_svr.predict(test_features)

  mse = mean_squared_error(test_labels[i], y_pred)

  # Compute the R-squared value
  r2 = r2_score(test_labels[i], y_pred)

  # Compute the mean absolute error
  mae = mean_absolute_error(test_labels[i], y_pred)

  metric['Emotion'] += [i]
  metric['MSE'] += [mse]
  metric['R2'] += [r2]
  metric['MAE'] += [mae]

  corr_coef = round(np.corrcoef(test_labels[i], y_pred)[0, 1], 2)

  cor_df['Emotion'] += [i]
  cor_df['Correlation'] += [corr_coef]

  print("Best SVR Model:", best_svr,' ' ,i)


Best SVR Model: SVR(C=10, epsilon=0.001)   valence
Best SVR Model: SVR(C=10)   energy
Best SVR Model: SVR(C=100)   tension
Best SVR Model: SVR(C=10, epsilon=0.001)   anger
Best SVR Model: SVR(C=10, epsilon=0.001)   fear
Best SVR Model: SVR(C=10)   happy
Best SVR Model: SVR(C=10)   sad
Best SVR Model: SVR(C=10, epsilon=0.001)   tender


In [12]:
cor_df = pd.DataFrame(cor_df)
cor_df

,Emotion,Correlation
0,valence,0.51
1,energy,0.62
2,tension,0.61
3,anger,0.62
4,fear,0.55
5,happy,0.30
6,sad,0.40
7,tender,0.54


In [13]:
metric = pd.DataFrame(metric)
metric

,Emotion,MSE,R2,MAE
0,valence,2.306649,0.248423,1.264866
1,energy,1.280739,0.279620,0.930960
2,tension,1.922786,0.336550,1.170880
3,anger,2.156085,0.293725,1.205446
4,fear,3.223688,0.298461,1.437490
5,happy,2.920041,0.087595,1.435348
6,sad,4.031465,0.100045,1.498948
7,tender,3.873427,0.194426,1.568062


In [ ]:
train_labels.columns.values

array(['valence', 'energy', 'tension', 'anger', 'fear', 'happy', 'sad',
       'tender'], dtype=object)